In [1]:
from stackapi import StackAPI
import os
import json
import time
import pandas as pd
import numpy as np

# 1 - Crawl tags meaning

In [2]:
SITE = StackAPI('stackoverflow',   key = "Y5FZk9DrTbRImUEfiQ5JJg((" , impose_throttling = True)

In [5]:
nodes = pd.read_csv("../DataSet StackOverflow/nodes.csv")

In [6]:
len(nodes)

13780

In [7]:
def check_tags_meaning(dt, nodes):
    not_found = []
    for i in nodes:
        try:
            tags = SITE.fetch(f"tags/{i}/wikis", filter = '!)rTkr_OQd(vR1j2O5o_q')
            if len(tags["items"]) > 0:
                    try:
                        x = tags["items"][0]["excerpt"]
                        dt[i] = tags["items"][0]["excerpt"]
                    except:
                        dt[i] = ""
                        not_found.append(i)
            else:
                dt[i] = ""
                print(i)
                not_found.append(i)
        except:
            dt[i]=""
            print(i)
            not_found.append(i)
    print(tags["quota_remaining"])
    return not_found, dt

In [8]:
dt = {}

In [20]:
not_found_900, dt = check_tags_meaning(dt, nodes["Id"][:9000])

beginthreadex
ohlc
apollo-gateway
nested-for-loop
python-interactive
kubebuilder
autofocus
webpack-5
angular-content-projection
avcaptureoutput
cleartext
webjob
tuf
azure-cosmosdb-sqlapi
nullish-coalescing
uinavigationbarappearance
ltk
customscrollview
tools.jar
id-generation
decltype-auto
reswift
ssms-2018
mat-sidenav
rust-result
viewbuilder
inline-scripting
swifter
component-diagram
tsdoc
formik-material-ui
completion
spark-checkpoint
pytest-xdist
google-java-format
devise-jwt
neighbours
ocmod
keymaps
springdoc-ui
mscoco
template-classes
dbutils
bitpay
tokio
vue-pwa
chrome-options
swagger-codegen-maven-plugin
safe-publication
cfthread
sbv
telegraf-plugins
ios-webkit-debug-proxy
rationale
iwyu
codesignkey
netcore.autoregisterdi
word-web-addins
pulsesecure
.net-standard-2.1
uiviewrepresentable
neopixel
ssis-connection-manager
flipper
standards-compliance
navigation-architecture
kubernetes-dns
google-map-react
onappear
shared-directory
protobuf-java
psscriptanalyzer
auto-renewable
celli

ckeditor5-react
azure-eventhub-capture
system-clock
react-native-safe-area-view
server.xml
facebook-ads
slowdown
parameter-pack
flask-testing
composable
spring-resttemplate
identification
type-synonyms
vue-render-function
trusted-vs-untrusted
uipointerinteraction
ssg
ratelimit
swashbuckle.aspnetcore
getcontent
c#-8.0
data-generation
multiple-value
mapkit-js
secretsmanager
solrclient
prometheus-node-exporter
outlet
react-native-linking
sitespeed
tika-server
nameservers
polymer-3.x
typecasting-operator
storage-access-api
tarantool-cartridge
cypress-cucumber-preprocessor
laravel-guard
error-messaging
google-cloud-ai
jobintentservice
clientcredential
power-analysis
flutter-in-app-purchase
copy-on-write
ipreviewhandler
self-supervised-learning
netlify-function
tf-hub
uint8list
gcp-ai-platform-training
spyon
react-sticky
combn
ios-permissions
visibilitychange
dbconnection
materialpageroute
graphql-mutation
android-fragment-manager
amazon-api-gateway
android-bubbles
gesture-state
elastic-clou

flutter-table
here-maps-rest
dryscrape
grecaptcha
extrafont
passport-jwt
express-generator
code-signing-entitlements
quantified-constraints
docker-command
azure-template
ids
unbuffered-queries
julia-dataframe
heremap-navigation
console-output
material-widget
connection-reset
react-on-rails
commandbar
saving-data
terraform-remote-state
cvxr
rapidapi
memoryview
feathers-authentication
flux.jl
standard-layout
in-app-promotions
vendor
rhel8
flutter-widget
django-jsonfield
scrapy-pipeline
tensorflow-android
primefaces-8.0
output-redirect
enable-shared-from-this
dynamic-frameworks
module.exports
cmake-gui
flask-table
docker-cli
r-glue
stdmove
numpy-indexing
elasticsearch-2.4
subtype
positional-parameter
gd-graph
kubernetes-networkpolicy
mediarecorder-api
initialization-order
relaxed-atomics
rust-bindgen
whitenoise
nestjs-passport
android-binding-adapter
full-expression
bull-queue
subapplication
nsviewcontrollerrepresentable
ebs
julia-plots
video-subtitles
leader-election
framer-motion
mapbox

In [23]:
len(dt)

8999

In [9]:
not_found_remaining, dt_remaining = check_tags_meaning(dt, nodes["Id"][9000:])

npm-package
deno-lint
mediadevices
smallrye
cursor-position
react-native-sqlite-storage
application-xml
jmenu
sidecar
react-aria
cucumber-spring
magento-2.3
aws-codeartifact
bazel-rules-nodejs
exponential-backoff
refund
language-history
launch-screen
ngrx-reducers
crashpad
tcpportsharing
flutteramplify
google-nearby-connections
babel-register
soundwaves
channels
drawtext
webpack-style-loader
job-control
multiple-cursor
mixed-integer-programming
memory-profiling
eclipse-iot
fastlane-gym
inspect-element
aws-fsx-windows
currency-pipe
verify
passport-local-mongoose
rgeo-shapefile
user-enrollment
injected-class-name
oidc-identity-brokering
http4s-circe
angular-errorhandler
identity-aware-proxy
haproxy-ingress
range-query
lob
telegraf-inputs-plugin
partial-ordering
mo-cap
fileprovider-extension
mediatorlivedata
gmailr
revenuecat
v8-profiler
typeclass-laws
pivotal-web-services
pytest-markers
payment-request-api
pass-by-rvalue-reference
stdmutex
android-motionlayout
geospark
shopify-template
p

flutter-http
microsoft-extensions-logging
getboundingclientrect
devicemotion
color-wheel
encoder-decoder
viewmodifier
share-extension
transport
toco
code-editor
amazon-cloudwatch-events
google-meet
angular-social-login
serverless-architecture
keyboardinterrupt
sections
opaque-types
carrier
comb
triplet
ihostedservice
hookrouter
mongodb-lookup
pen
lottie-android
deploying
gatsby-plugin
representation
node-html-pdf
stencil-component
google-speech-to-text-api
supertype
oauth2-proxy
flutter-tex
bottomappbar
repository-design
reqwest
appsettings
sectionindexer
highdpi
c#-7.3
podfile-lock
api-versioning
flair
hacker-news
polymorphic-functions
vscode-liveshare
runner
top-n
react-native-swiper
hikvision
ksuid
c#-record-type
sql-server-json
coverlet
laravel-upgrade
google-data-catalog
multistage-pipeline
android-architecture-lifecycle
android-update-app
lintr
google-cloud-api-gateway
azure-app-service-plans
remote-repository
elementwise-operations
xticks
pycocotools
orthanc-server
angular-route

In [12]:
len(dt_remaining)

4780

In [13]:
len(not_found_remaining)

806

In [10]:
tags = []
meaning = []
for i in dt_remaining:
    tags.append(i)
    meaning.append(dt_remaining[i])

In [43]:
len(meaning)

8999

In [11]:
new_df = pd.DataFrame({"Tags": tags,
                   "Meaning": meaning})
new_df.to_csv("../DataSet StackOverflow/partial_meaning_9000.csv", index=False)

In [3]:
df_partial_1 = pd.read_csv("../DataSet StackOverflow/partial_meaning_8999.csv")
df_partial_2 = pd.read_csv("../DataSet StackOverflow/partial_meaning_9000.csv")

In [4]:
def isNaN(string):
    return string != string

In [5]:
for i in df_partial_2:
    if isNaN(i):
        print(i)

In [6]:
tags = []
meaning = []

In [7]:
for i in df_partial_1.values:
    tags.append(i[0])
    if(isNaN(i[1])):
        meaning.append(" ")
    else:
        meaning.append(i[1])

In [8]:
for i in df_partial_2.values:
    tags.append(i[0])
    if(isNaN(i[1])):
        meaning.append(" ")
    else:
        meaning.append(i[1])

# 2 - Preprocess meanings

In [4]:
path = '../Dati wiki/pulito_nodes.csv'

In [5]:
df = pd.read_csv(path, error_bad_lines=False)

In [6]:
df

,Tag,Significati
0,fridaysforfuture,"School Strike for Climate Swedish, also known ..."
1,climatestrike,"School Strike for Climate Swedish, also known ..."
2,climateaction,Climate action (or climate change action) refe...
3,climatecrisis,Climate crisis is a term describing global war...
4,climatechange,Contemporary climate change includes both glo...
...,...,...
94,animals,"Animals are multicellular, eukaryotic organism..."
95,governo,Governo is a winemaking technique reportedly i...
96,emobility,EVs first came into existence in the mid-19th ...
97,chile,"Chile, officially the Republic of Chile, is a ..."


In [7]:
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import string

In [29]:
tags = df['Tag']
meaning = df['Significati']

In [16]:
def clean_text(corpus):
    clear_text = []
    for phrase in corpus:
        # Replace URL with the tag url
        phrase = re.sub("https*\S+", "url", phrase)
        # Remove strange characters
        # &quot => ""
        # &#39 => '
        # &amp; => &
        # &lt; => <
        # &gt; => >
        phrase = re.sub("&quot;|\r|\n\|&#39|&amp;|&lt;|&gt;", "", phrase)

        # Punctuaction removal
        phrase = re.sub('[%s]' % re.escape(string.punctuation), ' ', phrase)
        # Replace numbers with the tag number
        phrase = re.sub("\d+", "number ", phrase)
        # Convert to lowercase
        phrase = phrase.lower()
        # Replace the over spaces
        phrase = re.sub('\s{2,}', " ", phrase)
        clear_text.append([phrase])
    return clear_text

In [14]:
clean_text

<function __main__.clean_text(corpus)>

In [17]:
cleared_text = clean_text(meaning)

In [19]:
cleared_text

[['school strike for climate swedish also known variously as fridays for future fff youth for climate climate strike or youth strike for climate is an international movement of school students who skip friday classes to participate in demonstrations to demand action from political leaders to prevent climate change and for the fossil fuel industry to transition to renewable energy publicity and widespread organising began after swedish pupil greta thunberg staged a protest in august number outside of the swedish riksdag parliament holding a sign that read skolstrejk för klimatet school strike for climate a global strike on number march number gathered more than one million strikers in number number strikes organised in number countries on number may number in the second glo'],
 ['school strike for climate swedish also known variously as fridays for future fff youth for climate climate strike or youth strike for climate is an international movement of school students who skip friday clas

In [21]:
import networkx as nx
import itertools
from operator import itemgetter

In [23]:
G = nx.read_graphml("../Dataset FFF/Graph_data/Real_Network.graphml")

In [24]:
degree_dict = dict(G.degree(G.nodes()))
nx.set_node_attributes(G, degree_dict, 'degree')
sorted_degree = sorted(degree_dict.items(), key = itemgetter(1), reverse = True)

In [25]:
sample_nodes_597 = []
for i in sorted_degree:
    if i[1] > 50:
        sample_nodes_597.append(i[0])
print(len(sample_nodes_597))

257


In [26]:
sample_nodes_1057 = []
for i in sorted_degree:
    if i[1] > 30:
        sample_nodes_1057.append(i[0])
print(len(sample_nodes_1057))

511


In [27]:
def select_sample_meaning(sample, meaning):
    sampled_meaning = []
    sample_nodes = []
    for i in sample:
        for j in range(len(tags)):
            if tags[j]==i:
                sampled_meaning.append(i + " " + meaning[j][0])
                sample_nodes.append(i)
    return sampled_meaning, sample_nodes

In [30]:
sampled_meaning_597, sample_nodes_597 = select_sample_meaning(sample_nodes_597, cleared_text)
sampled_meaning_1057, sample_nodes_1057 = select_sample_meaning(sample_nodes_1057, cleared_text)

In [31]:
"""
for i in range(len(sample_nodes_1057)):
    if not isNaN(tags[i]):
        sampled_meaning_1057[i] = sample_nodes_1057[i] + " " + sampled_meaning_1057[i][0]
    else:
        sampled_meaning_1057[i] = "nan " + sampled_meaning_1057[i][0]
"""

'\nfor i in range(len(sample_nodes_1057)):\n    if not isNaN(tags[i]):\n        sampled_meaning_1057[i] = sample_nodes_1057[i] + " " + sampled_meaning_1057[i][0]\n    else:\n        sampled_meaning_1057[i] = "nan " + sampled_meaning_1057[i][0]\n'

In [32]:
word_tokens = []
for phrase in sampled_meaning_597:
    word_tokens.append(phrase.split())
stop_words = stopwords.words('english')
no_sw_597=[]
for phrase in word_tokens:
    no_sw_tmp=[]
    for word in phrase:
        if word not in stop_words:
            no_sw_tmp.append(word)
    no_sw_597.append(no_sw_tmp)

In [33]:
word_tokens = []
for phrase in sampled_meaning_1057:
    word_tokens.append(phrase.split())
stop_words = stopwords.words('english')
no_sw_1057=[]
for phrase in word_tokens:
    no_sw_tmp=[]
    for word in phrase:
        if word not in stop_words:
            no_sw_tmp.append(word)
    no_sw_1057.append(no_sw_tmp)

In [34]:
cleared_meaning_597 = []
for i in no_sw_597:
    new_s = ""
    for j in i:
        new_s = new_s + " " + j
    cleared_meaning_597.append(new_s)

In [35]:
cleared_meaning_1057 = []
for i in no_sw_1057:
    new_s = ""
    for j in i:
        new_s = new_s + " " + j
    cleared_meaning_1057.append(new_s)

In [36]:
df = pd.DataFrame({"Tags": sample_nodes_597, "Meaning" : cleared_meaning_597})
df.to_csv("nodes_597_meaning.csv", index=False)

In [37]:
df = pd.DataFrame({"Tags": sample_nodes_1057, "Meaning" : cleared_meaning_1057})
df.to_csv("nodes_1057_meaning.csv", index=False)

In [38]:
from gensim.models import Word2Vec

In [39]:
word2vec_597 = Word2Vec(no_sw_597, min_count=1)

In [40]:
len(word2vec_597.wv)

1323

In [41]:
def check_isin(word2vec, data):
    not_in = []
    for i in data:
        try:
            word2vec.wv[i]
        except:
            not_in.append(i)
    print(len(not_in))

In [42]:
word2vec_1057 = Word2Vec(no_sw_1057, min_count=1)

In [43]:
len(word2vec_1057.wv)

1538

In [45]:
G = nx.read_graphml("../Dataset FFF/Graph_data/Real_Network.graphml")

# 3 - Store word2vec embedding

In [46]:
from linkpred.evaluation import Pair
import pickle

In [47]:
sample_nodes_1057.remove("this")

ValueError: list.remove(x): x not in list

In [48]:
H_597 = G.subgraph(sample_nodes_597)
H_1057 = G.subgraph(sample_nodes_1057)

In [49]:
first = []
second = []
for i in H_597.edges():
    first.append(word2vec_597.wv[i[0]])
    second.append(word2vec_597.wv[i[1]])
new_df = pd.DataFrame({"first":first, "second":second})
new_df.reset_index(inplace=True, drop=True)
new_df.to_pickle("embedding_pickle_597.csv")

In [50]:
first = []
second = []
for i in H_1057.edges():
    first.append(word2vec_1057.wv[i[0]])
    second.append(word2vec_1057.wv[i[1]])
new_df = pd.DataFrame({"first":first, "second":second})
new_df.reset_index(inplace=True, drop=True)
new_df.to_pickle("embedding_pickle_1057.csv")

In [51]:
word2vec_597.save("word2vec_597.model")

In [52]:
word2vec_1057.save("word2vec_1057.model")

In [53]:
nodes = list(H_597.nodes())
universe_597 = set([Pair(i) for i in itertools.product(nodes, nodes) if i[0]!=i[1]])

In [54]:
nodes = list(H_1057.nodes())
universe_1057 = set([Pair(i) for i in itertools.product(nodes, nodes) if i[0]!=i[1]])

In [55]:
with open('universe_1057.pickle', 'wb') as f:
    pickle.dump(universe_1057, f)
with open('universe_597.pickle', 'wb') as f:
    pickle.dump(universe_597, f)

In [56]:
first = []
second = []
for i in H_597.edges():
    first.append(i[0])
    second.append(i[1])

In [57]:
new_df = pd.DataFrame({"first":first, "second":second})
new_df.reset_index(inplace=True, drop=True)
new_df.to_pickle("edges_original_597.csv")

In [58]:
first = []
second = []
for i in H_1057.edges():
    first.append(i[0])
    second.append(i[1])

In [59]:
new_df = pd.DataFrame({"first":first, "second":second})
new_df.reset_index(inplace=True, drop=True)
new_df.to_pickle("edges_original_1057.csv")

In [60]:
nx.write_graphml(H_597,"H_597.graphml")
nx.write_graphml(H_1057,"H_1057.graphml")

-------------------------------------